<a href="https://colab.research.google.com/github/serrabaysal/auto-ml-course-notes/blob/main/6aral%C4%B1k2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Auto ML
1.   Auto-sklearn:Auto ML scikit learn kütüphanesini sarmalayarak tam otomatik bir çözüm yöntemi sunar.Meta learning:rasgele başlamak yerine başarı ihtimali yüksek bir noktadan başlar. BU süre kısıtlı olduğundan çok büyük bir avantaj sağlar. Bayes optimizasyonu Ensemble Selection tarzı baya yöntemler denenir.
Topluluk öğrenmesi. Kısa sürede yüksek performans ,otomatik ensemble kurması başarıyı artırır,küçük\orta veri setleri için mükemel,scikit-learn ile tam uyumlu arayüz
1.   TPOT:Tree Based Optimization:Başlangıç,Değerlendirme,Üreme. TPOT daha basit olan  modeli seçer.Python kodu çıktısı verdiği için modeli ürünleştirmek daha kolaydır. Süre kısıtlamasına uymakta zorlanabilir.
2.   H2O.ai






In [ ]:
 #TPOT
 !pip install -q tpot

 from tpot import TPOTClassifier
 from sklearn.datasets import load_breast_cancer
 from sklearn.model_selection import train_test_split
 from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
 import time

 #1) Veri seti
 data=load_breast_cancer()
 X,y=data.data, data.target
 class_names=data.target_names
 print("Sınıflar:",class_names)

#2) Train\Test bölme
X_train,X_test,y_train,y_test = train_test_split(
    X,y,
    test_size=0.2,
    random_state=42,
    stratify=y
)
print("Train shape:",X_train.shape)
print("Test  shape:",X_test.shape)

#3)TPOTClassifier (sürüm uyumu için sadece çekirdek parametreler)
#NOT: scorin,max_time_mins

tpot=TPOTClassifier(
      generations=5,       #evrim nesli
      population_size=20,  #her nesilden kaç pipeline denenecek.
      cv=5,                #5-fold CV
      random_state=42,

      )

# 4) EĞİT (süre ölçümü ile)
start = time.time()
tpot.fit(X_train, y_train)
end = time.time()
print(f"\nEğitim süresi: {end - start:.2f} sn)")

# 5) Tahmin ve temel metrikler
y_pred = tpot.predict(X_test)
print("\n========== Temel Metrikler ==========")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred)}")
print("\n====================================")
print("Sınıflandırma Raporu:\n")
print(classification_report(y_test, y_pred, target_names=class_names))
print("Confusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))

# 6) EN İYİ PIPELINE'I GÖR
print("\n====== EN İYİ PIPELINE (tpot.fitted_pipeline_) ======\n")
print(tpot.fitted_pipeline_)

# 7) EN İYİ MODELİN DETAYLARI (son adımdaki tahminleyici)
best_model = tpot.fitted_pipeline_.steps[-1][1]
print("\n====== EN İYİ MODEL (Son Adım) ======")
print(f"\nModel tipi: {type(best_model)}")

# Hiperparametrelerin tamamını yazdır
print("\nModel hiperparametreleri:\n")
for k, v in best_model.get_params().items():
    print(f"{k}: {v}")

# 8) İstersen pipeline'ı .py dosyasına da kaydedebilirsin
from pathlib import Path
export_path = Path("tpot_breast_cancer_pipeline.py")
tpot.export(str(export_path))
print(f'\nPipeline kodu "{export_path.name}" dosyasına kaydedildi.')

Sınıflar: ['malignant' 'benign']
Train shape: (455, 30)
Test  shape: (114, 30)


/usr/local/lib/python3.12/dist-packages/tpot/tpot_estimator/estimator.py:458: UserWarning: Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.
  warnings.warn("Both generations and max_time_mins are set. TPOT will terminate when the first condition is met.")
INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:34379
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:45351'
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:35241 name: 0
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:35241
INFO:distributed.core:Starting established connection to tcp://127.0.0.1


Eğitim süresi: 1027.49 sn)

========== Temel Metrikler ==========
Test Accuracy: 0.9824561403508771

Sınıflandırma Raporu:

              precision    recall  f1-score   support

   malignant       1.00      0.95      0.98        42
      benign       0.97      1.00      0.99        72

    accuracy                           0.98       114
   macro avg       0.99      0.98      0.98       114
weighted avg       0.98      0.98      0.98       114

Confusion Matrix:

[[40  2]
 [ 0 72]]

====== EN İYİ PIPELINE (tpot.fitted_pipeline_) ======

Pipeline(steps=[('maxabsscaler', MaxAbsScaler()),
                ('selectpercentile',
                 SelectPercentile(percentile=74.1105445568588)),
                ('featureunion-1',
                 FeatureUnion(transformer_list=[('featureunion',
                                                 FeatureUnion(transformer_list=[('binarizer',
                                                                                 Binarizer(threshold=0.01264

AttributeError: 'TPOTClassifier' object has no attribute 'export'

In [ ]:
# ============================================================
# H2O AutoML - Breast Cancer Örneği (Colab için)
# ============================================================

# 1) Kurulum
!pip install -q h2o

# 2) Kütüphaneler
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import pandas as pd

# 3) H2O sunucusunu başlat
h2o.init()

# 4) Sklearn breast cancer veri setini yükle → pandas DataFrame
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name="target")

print("Sınıflar:", data.target_names)

# 5) Train / test böl
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

train_df = X_train.copy()
train_df["target"] = y_train

test_df = X_test.copy()
test_df["target"] = y_test

print("Train shape:", train_df.shape)
print("Test  shape:", test_df.shape)

# 6) pandas → H2OFrame dönüşümü
train_h2o = h2o.H2OFrame(train_df)
test_h2o  = h2o.H2OFrame(test_df)

# Hedef değişkeni kategorik (binari sınıflandırma) olarak ayarla
label = "target"
train_h2o[label] = train_h2o[label].asfactor()
test_h2o[label]  = test_h2o[label].asfactor()

features = [col for col in train_h2o.columns if col != label]

# 7) H2O AutoML tanımı ve eğitimi
aml = H2OAutoML(
    max_runtime_secs=300,   # toplam arama süresi (saniye)
    max_models=20,          # en fazla 20 model dene
    nfolds=5,               # 5-fold CV
    balance_classes=True,   # dengesiz sınıflar için
    sort_metric="AUC",      # sıralama metriği
    seed=42
)

aml.train(
    x=features,
    y=label,
    training_frame=train_h2o
)

# 8) Liderlik tablosu (hangi modeller denenmiş, skorları neler)
lb = aml.leaderboard
print("===== Leaderboard (İlk 10 model) =====")
print(lb.head(10))

# 9) Test seti üzerinde performans
leader = aml.leader
perf = leader.model_performance(test_h2o)

print("\n===== Test Performansı (Leader Model) =====")
print(perf)

# Özet metrikleri ayrı yazdırmak istersen:
print("\nAUC:", perf.auc())
print("Logloss:", perf.logloss())
print("Accuracy (default threshold):", perf.accuracy()[0][1])

# 10) En iyi modelin tipini göster
print("\nEn iyi model ID   :", leader.model_id)
print("Model tipi (algo):", leader.algo)

# İşin bittiğinde H2O'yu kapatmak istersen:
# h2o.shutdown(prompt=False)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.0/266.0 MB 2.8 MB/s eta 0:00:00
Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17.0.16" 2025-07-15; OpenJDK Runtime Environment (build 17.0.16+8-Ubuntu-0ubuntu122.04.1); OpenJDK 64-Bit Server VM (build 17.0.16+8-Ubuntu-0ubuntu122.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.12/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpv1qp1unn
  JVM stdout: /tmp/tmpv1qp1unn/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpv1qp1unn/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.9
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_unknownUser_mt184b
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.168 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Sınıflar: ['malignant' 'benign']
Train shape: (455, 31)
Test  shape: (114, 31)
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
===== Leaderboard (İlk 10 model) =====
model_id                                                      auc    logloss     aucpr    mean_per_class_error      rmse        mse
DeepLearning_grid_1_AutoML_1_20251206_125342_model_1     0.996553  0.0811827  0.997847               0.0229102  0.142218  0.020226
StackedEnsemble_AllModels_1_AutoML_1_20251206_125342     0.996202  0.0637262  0.997548               0.0175955  0.129614  0.0167998
StackedEnsemble_BestOfFamily_1_AutoML_1_20251206_125342  0.994964  0.0738872  0.996658               0.0234778  0.139577  0.0194816
GBM_2_AutoML_1_20251206_125342                           0.994943  0.0

In [ ]:
# ============================================================
# AutoGluon - Breast Cancer (Tabular) Örneği - Colab için
# ============================================================

# 1) Kurulum (Colab için)
!pip install -U pip >/dev/null
!pip install autogluon.tabular -q

# 2) Gerekli kütüphaneler
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

# 3) Veri setini yükle (sklearn → pandas DataFrame)
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name="target")

# Sınıf isimlerini saklayalım (0: malignant, 1: benign vs.)
class_names = data.target_names
print("Sınıflar:", class_names)

# 4) Train / Test bölme
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

train_df = X_train.copy()
train_df["target"] = y_train

test_df = X_test.copy()
test_df["target"] = y_test  # AutoGluon değerlendirme için isterse kullanır

print("Train shape:", train_df.shape)
print("Test shape :", test_df.shape)

# 5) AutoGluon TabularPredictor tanımı
label = "target"

predictor = TabularPredictor(
    label=label,
    problem_type="binary",      # iki sınıflı problem
    eval_metric="roc_auc"       # daha dengeli bir metrik
).fit(
    train_data=train_df,
    time_limit=300,             # 5 dakika arama süresi
    presets="medium_quality_faster_train"  # hız/performans dengesi
)

# 6) Tahmin ve değerlendirme
y_pred = predictor.predict(test_df)

print("\n============================")
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("============================\n")

print("Sınıflandırma Raporu:\n")
print(classification_report(y_test, y_pred, target_names=class_names))

print("Confusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))

# 7) AutoGluon liderlik tablosu (hangi modelleri denemiş, hangisi en iyi)
print("\n===== Leaderboard =====")
leaderboard_df = predictor.leaderboard(test_df, silent=True)
display(leaderboard_df)

# 8) En iyi modeli ve tipini göster
best_model_name = leaderboard_df.loc[0, 'model']
print("\nEn iyi model:", best_model_name)
# Removed the problematic get_model_info call and using best_model_name directly
print("Model tipi:", best_model_name)


No path specified. Models will be saved in: "AutogluonModels/ag-20251206_131600"
Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Memory Avail:       9.48 GB / 12.67 GB (74.9%)
Disk Space Avail:   68.73 GB / 107.72 GB (63.8%)
Presets specified: ['medium_quality_faster_train']
Using hyperparameters preset: hyperparameters='default'


Sınıflar: ['malignant' 'benign']
Train shape: (455, 31)
Test shape : (114, 31)


Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "/content/AutogluonModels/ag-20251206_131600"
Train Data Rows:    455
Train Data Columns: 30
Label Column:       target
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    9712.67 MB
	Train Data (Original)  Memory Usage: 0.10 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data 


Test Accuracy: 0.956140350877193

Sınıflandırma Raporu:

              precision    recall  f1-score   support

   malignant       0.91      0.98      0.94        42
      benign       0.99      0.94      0.96        72

    accuracy                           0.96       114
   macro avg       0.95      0.96      0.95       114
weighted avg       0.96      0.96      0.96       114

Confusion Matrix:

[[41  1]
 [ 4 68]]

===== Leaderboard =====


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetFastAI,0.997354,0.997936,roc_auc,0.013410,0.006680,0.669100,0.013410,0.006680,0.669100,1,True,7
1,NeuralNetTorch,0.995040,0.998452,roc_auc,0.021260,0.018183,6.374279,0.021260,0.018183,6.374279,1,True,9
2,ExtraTreesGini,0.994213,0.993292,roc_auc,0.100910,0.086562,0.751062,0.100910,0.086562,0.751062,1,True,5
3,LightGBMXT,0.993717,0.998452,roc_auc,0.009150,0.002606,0.908015,0.009150,0.002606,0.908015,1,True,1
4,RandomForestEntr,0.993386,0.992518,roc_auc,0.100493,0.079342,1.287001,0.100493,0.079342,1.287001,1,True,4
5,WeightedEnsemble_L2,0.993056,0.999484,roc_auc,0.033270,0.021492,7.400361,0.002860,0.000702,0.118068,2,True,11
6,XGBoost,0.992725,0.996388,roc_auc,0.018900,0.004949,0.409126,0.018900,0.004949,0.409126,1,True,8
7,LightGBMLarge,0.992394,0.990196,roc_auc,0.011714,0.003110,2.329763,0.011714,0.003110,2.329763,1,True,10
8,ExtraTreesEntr,0.992229,0.992260,roc_auc,0.098770,0.088403,0.756433,0.098770,0.088403,0.756433,1,True,6
9,RandomForestGini,0.990741,0.992002,roc_auc,0.100531,0.075950,1.145872,0.100531,0.075950,1.145872,1,True,3



En iyi model: NeuralNetFastAI


AttributeError: 'TabularPredictor' object has no attribute 'get_model_info'